## Code Setup

### Loading packages, setting up gpu, loading models

In [4]:
# required packages
from transformers import AutoProcessor, CLIPTextModel, CLIPVisionModel, PreTrainedModel
import torch
import torch_directml
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
from PIL import Image
import glob
import time
from sklearn.metrics import euclidean_distances
from sklearn.decomposition import IncrementalPCA, PCA
from sklearn.linear_model import LinearRegression
from scipy.spatial.distance import squareform
from scipy.stats import spearmanr
from scipy.stats import pearsonr as corr
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
#from nilearn import datasets
#from nilearn import plotting
from typing import List, Dict, Tuple

c:\Users\LukeKorthals\Desktop\Python Repositories\algonauts-2023\.conda\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


We used an AMD RX 5700XT GPU and AMD Ryzen 5 3600XT CPU to run this code. To enable GPU support we relied on torch_directml == 0.1.13.1.dev230413.

If you want to run this notebook on a cuda device, set AMD to False.

In [2]:
# Setup cuda device
global device
AMD = True
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
if AMD:
    device = torch_directml.device()

We downloaded pretrained CLIPModels and the CLIPProcessor from huggingface. 

In [3]:
global vis_model, txt_model, processor
vis_model = CLIPVisionModel.from_pretrained("openai/clip-vit-base-patch32")
txt_model = CLIPTextModel.from_pretrained("openai/clip-vit-base-patch32")
processor = AutoProcessor.from_pretrained("openai/clip-vit-base-patch32")
vis_model.eval()
txt_model.eval()

Some weights of the model checkpoint at openai/clip-vit-base-patch32 were not used when initializing CLIPVisionModel: ['text_model.encoder.layers.1.self_attn.q_proj.weight', 'text_model.encoder.layers.7.mlp.fc2.weight', 'text_model.encoder.layers.9.self_attn.v_proj.bias', 'text_model.encoder.layers.6.self_attn.k_proj.weight', 'text_model.encoder.layers.5.layer_norm2.weight', 'text_model.embeddings.position_ids', 'text_model.encoder.layers.10.mlp.fc2.weight', 'text_model.encoder.layers.6.self_attn.out_proj.bias', 'text_model.encoder.layers.11.layer_norm1.weight', 'text_model.encoder.layers.1.layer_norm1.weight', 'text_model.encoder.layers.7.layer_norm2.bias', 'text_model.encoder.layers.9.self_attn.q_proj.weight', 'text_model.encoder.layers.10.self_attn.v_proj.bias', 'text_projection.weight', 'text_model.encoder.layers.8.self_attn.q_proj.bias', 'text_model.encoder.layers.4.self_attn.out_proj.weight', 'text_model.encoder.layers.0.self_attn.q_proj.bias', 'text_model.encoder.layers.1.self_a

CLIPTextModel(
  (text_model): CLIPTextTransformer(
    (embeddings): CLIPTextEmbeddings(
      (token_embedding): Embedding(49408, 512)
      (position_embedding): Embedding(77, 512)
    )
    (encoder): CLIPEncoder(
      (layers): ModuleList(
        (0): CLIPEncoderLayer(
          (self_attn): CLIPAttention(
            (k_proj): Linear(in_features=512, out_features=512, bias=True)
            (v_proj): Linear(in_features=512, out_features=512, bias=True)
            (q_proj): Linear(in_features=512, out_features=512, bias=True)
            (out_proj): Linear(in_features=512, out_features=512, bias=True)
          )
          (layer_norm1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (mlp): CLIPMLP(
            (activation_fn): QuickGELUActivation()
            (fc1): Linear(in_features=512, out_features=2048, bias=True)
            (fc2): Linear(in_features=2048, out_features=512, bias=True)
          )
          (layer_norm2): LayerNorm((512,), eps=1e-05, ele

### Classes & Functions

#### ImageDataset & TextDataset

The ImageDataset and TextDataset classes are used to create datasets for torch dataloaders.
The ImageDataset is used for the images and the TextDataset for the captions of the coco images.

In [4]:
# Dataset Classes for Batching
class ImageDataset(Dataset):
    def __init__(self, image_list, processor):
        self.image_list = image_list
        self.processor = processor

    def __len__(self):
        return len(self.image_list)

    def __getitem__(self, idx):
        image = Image.open(self.image_list[idx])
        image = self.processor(images=image, return_tensors="pt", padding=True)
        return image["pixel_values"].squeeze()

class TextDataset(Dataset):
    def __init__(self, text, max_length, processor):
        self.text = processor(text=text, return_tensors="pt", padding="max_length", max_length=max_length)

    def __len__(self):
        return len(self.text["input_ids"])

    def __getitem__(self, idx):
        return self.text["input_ids"][idx]

#### Subject Class

The Subject class is initialized with a valid subject id (e.g., "subj01"). It stores all relevant paths and can load the data for the given subject.

In [6]:
class Subject:
    """Class to access all relevant data for a given subject"""
    def __init__(self, subject="subj01"):
        assert subject in ["subj01", "subj02", "subj03", "subj04", "subj05", "subj06", "subj07", "subj08",], "Invalid subject"
        self.subject = subject
        self.data_dir = "data/algonauts_2023_challenge_data"
        self.training_images_dir = f"{self.data_dir}/{subject}/training_split/training_images"
        self.test_images_dir = f"{self.data_dir}/{subject}/test_split/test_images"
        self.training_fmri_dir = f"{self.data_dir}/{subject}/training_split/training_fmri"
        self.roi_masks_dir = f"{self.data_dir}/{subject}/roi_masks"
        self.submission_dir = f"algonauts_2023_challenge_submission"
        # Load these as needed
        self.train_img_list = None
        self.test_img_list = None
        self.train_cap_list = None
        self.test_cap_list = None
        self.lh_fmri = None
        self.rh_fmri = None
        self.lh_roi_masks = None
        self.rh_roi_masks = None
        self.roi_name_maps = None
        self.lh_challenge_rois = None
        self.rh_challenge_rois = None
        self.train_img_dataloader = None
        self.test_img_dataloader = None
        self.train_cap_dataloader = None
        self.test_cap_dataloader = None            
        
    def load_image_paths(self) -> None:
        """Loads the image paths from the training and test directories"""
        self.train_img_list = glob.glob(f"{self.training_images_dir}/*.png")
        self.train_img_list.sort()
        self.test_img_list = glob.glob(f"{self.test_images_dir}/*.png")
        self.test_img_list.sort()
        print(f"Training images: {len(self.train_img_list)}")
        print(f"Test images: {len(self.test_img_list)}")

    def load_captions(self) -> None:
        """Loads and matches the captions from the csv file"""
        if self.train_img_list is None:
            self.load_image_paths()
        train_cap_file = pd.read_csv(f'{self.data_dir}/algonauts_2023_caption_data.csv')
        img_match = [int(i[-9:-4]) for i in self.train_img_list]
        self.train_cap_list = train_cap_file[(train_cap_file['subject'] == self.subject) & (train_cap_file['nsdId'].isin(img_match))]['caption'].tolist()
        self.test_cap_list = train_cap_file[(train_cap_file['subject'] == self.subject) & (~train_cap_file['nsdId'].isin(img_match))]['caption'].tolist()
        print(f"Training captions: {len(self.train_cap_list)}")
        print(f"Test captions: {len(self.test_cap_list)}")
    
    def load_neural_data(self) -> None:
        """Loads the neural data from the .npy files"""
        self.lh_fmri = np.load(f"{self.training_fmri_dir}/lh_training_fmri.npy")
        self.rh_fmri = np.load(f"{self.training_fmri_dir}/rh_training_fmri.npy")
        print(f"Left hemisphere neural data loaded. Shape: {self.lh_fmri.shape}")
        print(f"Right hemisphere neural data loaded. Shape: {self.rh_fmri.shape}")

    def create_dataloaders(self, processor, batch_size) -> None:
        """Creates the dataloaders for the images and captions"""
        if self.train_img_list is None:
            self.load_image_paths()
        if self.train_cap_list is None:
            self.load_captions()
        max_caption_len = processor(text=self.train_cap_list + self.test_cap_list, return_tensors="pt", padding=True)["input_ids"].shape[1]   
        train_txt_dataset = TextDataset(self.train_cap_list, max_caption_len, processor)
        test_txt_dataset = TextDataset(self.test_cap_list, max_caption_len, processor)
        train_img_dataset = ImageDataset(self.train_img_list, processor)
        test_img_dataset = ImageDataset(self.test_img_list, processor)
        self.train_img_dataloader = DataLoader(train_img_dataset, batch_size=batch_size, shuffle=False)
        self.test_img_dataloader = DataLoader(test_img_dataset, batch_size=batch_size, shuffle=False)
        self.train_txt_dataloader = DataLoader(train_txt_dataset, batch_size=batch_size, shuffle=False)
        self.test_txt_dataloader = DataLoader(test_txt_dataset, batch_size=batch_size, shuffle=False)
        print(f"Train image dataloader: {len(self.train_img_dataloader)} batches")
        print(f"Test image dataloader: {len(self.test_img_dataloader)} batches")
        print(f"Train caption dataloader: {len(self.train_txt_dataloader)} batches")
        print(f"Test caption dataloader: {len(self.test_txt_dataloader)} batches")

    def load_challenge_rois(self) -> None:
        """Loads the challenge rois from the .npy files"""
        # Load the ROI classes mapping dictionaries
        roi_mapping_files = ['mapping_prf-visualrois.npy', 'mapping_floc-bodies.npy',
            'mapping_floc-faces.npy', 'mapping_floc-places.npy',
            'mapping_floc-words.npy', 'mapping_streams.npy']
        self.roi_name_maps = []
        for r in roi_mapping_files:
            self.roi_name_maps.append(np.load(f"{self.roi_masks_dir}/{r}", allow_pickle=True).item())

        # Load the ROI brain surface maps
        lh_challenge_roi_files = ['lh.prf-visualrois_challenge_space.npy',
            'lh.floc-bodies_challenge_space.npy', 'lh.floc-faces_challenge_space.npy',
            'lh.floc-places_challenge_space.npy', 'lh.floc-words_challenge_space.npy',
            'lh.streams_challenge_space.npy']
        rh_challenge_roi_files = ['rh.prf-visualrois_challenge_space.npy',
            'rh.floc-bodies_challenge_space.npy', 'rh.floc-faces_challenge_space.npy',
            'rh.floc-places_challenge_space.npy', 'rh.floc-words_challenge_space.npy',
            'rh.streams_challenge_space.npy']
        self.lh_challenge_rois = []
        self.rh_challenge_rois = []
        for r in range(len(lh_challenge_roi_files)):
            self.lh_challenge_rois.append(np.load(f"{self.roi_masks_dir}/{lh_challenge_roi_files[r]}"))
            self.rh_challenge_rois.append(np.load(f"{self.roi_masks_dir}/{rh_challenge_roi_files[r]}"))

    def load_roi_masks(self, roi="V1v", hemisphere="lh"):
        valid_roi = ["V1v", "V1d", "V2v", "V2d", "V3v", 
                     "V3d", "hV4", "EBA", "FBA-1", "FBA-2", 
                     "mTL-bodies", "OFA", "FFA-1", "FFA-2", 
                     "mTL-faces", "aTL-faces", "OPA", "PPA", 
                     "RSC", "OWFA", "VWFA-1", "VWFA-2", "mfs-words", 
                     "mTL-words", "early", "midventral", "midlateral", 
                     "midparietal", "ventral", "lateral", "parietal",
                     "all-vertices"]
        valid_hemisphere = ["lh", "rh"]
        assert roi in valid_roi, "Invalid ROI"
        assert hemisphere in valid_hemisphere, "Invalid hemisphere"

        # Define the ROI class based on the selected ROI
        if roi in ["V1v", "V1d", "V2v", "V2d", "V3v", "V3d", "hV4"]:
            roi_class = 'prf-visualrois'
        elif roi in ["EBA", "FBA-1", "FBA-2", "mTL-bodies"]:
            roi_class = 'floc-bodies'
        elif roi in ["OFA", "FFA-1", "FFA-2", "mTL-faces", "aTL-faces"]:
            roi_class = 'floc-faces'
        elif roi in ["OPA", "PPA", "RSC"]:
            roi_class = 'floc-places'
        elif roi in ["OWFA", "VWFA-1", "VWFA-2", "mfs-words", "mTL-words"]:
            roi_class = 'floc-words'
        elif roi in ["early", "midventral", "midlateral", "midparietal", "ventral", "lateral", "parietal"]:
            roi_class = 'streams'
        else:
            roi_class = roi
        roi_class_dir = f"{hemisphere}.{roi_class}_fsaverage_space.npy"
        roi_map_dir = f"mapping_{roi_class}.npy"
        fsaverage_roi_class = np.load(f"{self.roi_masks_dir}/{roi_class_dir}")
        roi_map = None
        if roi != "all-vertices":
            roi_map = np.load(f"{self.roi_masks_dir}/{roi_map_dir}", allow_pickle=True).item()
        return fsaverage_roi_class, roi_map

#### CLIPFeatureExtractor Class

The CLIPFeatureExtractor class is used to extract the hidden states from a clip model.

In [7]:
class CLIPFeatureExtractor():
    """Extracts the features from hidden states of a CLIP model."""
    def __init__(
            self, 
            idxs: list = [i for i in range(13)], # hidden layer indices to extract features from. Standard CLIP has an embedding layer and 12 transformer layers.
            last_hidden_layer: bool = False, # whether to extract features from the last hidden layer
            model: PreTrainedModel = None, # CLIP model
            dataloader: DataLoader = None, # dataloader for batching
            ) -> None:
        self.idxs = idxs
        self.last_hidden_layer = last_hidden_layer
        self.generate_feature_dict()
        if self.last_hidden_layer:
            self.idxs.append(13) # adds an additional idx to allow for loop zip()
        self.model = model
        self.dataloader = dataloader
        print(f"idxs: {self.idxs}")
        print(f"feature dict keys: {self.feature_dict.keys()}")
    
    def generate_feature_dict(self) -> None:
        """Generates a feature dict according to the idxs and last_hidden_layer attributes."""
        feature_dict = {}
        for idx in self.idxs:
            if idx == 0:
                feature_dict["Embedding Layer"] = None
            else:
                feature_dict[f"Transformer Layer {idx}"] = None
        if self.last_hidden_layer:
            feature_dict["Final Layer"] = None
        self.feature_dict = feature_dict
    
    def concat_features(self, features: dict) -> None:
        """Adds extracted features to the feature dict.
        Args:
            features: features extracted from the output of a CLIP model"""
        keys = list(self.feature_dict.keys())
        # check if feature_dict is empty
        if self.feature_dict[keys[0]] is None:
            self.feature_dict = features
        else:
            for key in keys:
                self.feature_dict[key] = np.concatenate((self.feature_dict[key], features[key]), axis=0)

    def extract_raw_features(self, output) -> None: 
        """Extracts features from the hidden states of a CLIP model and concates them to the feature_dict.
        Args:
            output: output of a CLIP model
        """
        features = {}
        for idx, key in zip(self.idxs, self.feature_dict.keys()):
            if key == "Final Layer":
                features[key] = output.last_hidden_state.cpu().detach().numpy()
            else:
                features[key] = output.hidden_states[idx].cpu().detach().numpy()
        self.concat_features(features)
    
    def extract_raw_features_from_model(self) -> None:
        """Runs the CLIP model on the dataloader and extracts features from the hidden states."""
        self.model = self.model.to(device)
        with torch.no_grad():
            for batch in tqdm(self.dataloader):
                batch = batch.to(device)
                output = self.model(batch, output_hidden_states=True)
                self.extract_raw_features(output)
                batch = None # clear batch from memory
                output = None # clear output from memory
        self.model = self.model.to("cpu")        

#### KFoldProcedure & KFold Classes

The KFoldProcedure class is used to define a procedure that is supposed to be executed during each fold of the k-fold validation. 
It can be supplied to a KFold class which executes its run() function on all folds.

In [33]:
class KFoldProcedure:
    """This class is used to define a procedure that is run on each fold of a k-fold cross validation."""
    def __init__(self) -> None:
        assert isinstance(self.model_name, str) and len(self.model_name) > 0, "Please define a model name as part of the KFold Procedure."
        assert isinstance(self.description, str ) and len(self.description) > 0 , "Please define a description as part of the KFold Procedure."

    def prepare(self) -> None:
        """Operations that should be executed before the fold loop"""
        raise NotImplementedError

    def run(self, train_idxs: np.ndarray, val_idxs: np.ndarray) -> Dict[str, Dict[str, np.ndarray]]:
        """This should return a dict of correlations.
        dict format: {"layer": {"lh": np.ndarray, "lh": np.ndarray}}"""
        raise NotImplementedError
    
    def return_idxs(self):
        """Returns idxs to create folds in the KFold class."""
        raise NotImplementedError
    
    def return_roi_names(self) -> List[str]:
        """Required for the plot function in the KFold class."""
        return self.roi_names    
    
    def return_model_name_and_description(self) -> Tuple[str, str]:
        return self.model_name, self.description

    def calculate_correlations(self, lh_pred, rh_pred, lh_fmri, rh_fmri) -> Tuple[np.ndarray, np.ndarray]:
        """Calculate correlation between prediction and fmri activation"""
        lh_correlation = np.zeros(lh_pred.shape[1])
        for v in tqdm(range(lh_pred.shape[1])):
            lh_correlation[v] = corr(lh_pred[:,v], lh_fmri[:,v])[0]
        # Right hemisphere
        rh_correlation = np.zeros(rh_pred.shape[1])
        for v in tqdm(range(rh_pred.shape[1])):
            rh_correlation[v] = corr(rh_pred[:,v], rh_fmri[:,v])[0]
        return lh_correlation, rh_correlation

    def calculate_median_correlations(self, lh_correlation, rh_correlation) -> Tuple[np.ndarray, np.ndarray]:
        """Calculate median correlation for each ROI."""
        # Select the correlation results vertices of each ROI
        lh_challange_rois = self.lh_challenge_rois
        rh_challange_rois = self.rh_challenge_rois
        self.roi_names = []
        lh_roi_correlation = []
        rh_roi_correlation = []
        for r1 in range(len(lh_challange_rois)):
            for r2 in self.roi_name_maps[r1].items():
                if r2[0] != 0: # zeros indicate to vertices falling outside the ROI of interest
                    self.roi_names.append(r2[1])
                    lh_roi_idx = np.where(lh_challange_rois[r1] == r2[0])[0]
                    rh_roi_idx = np.where(rh_challange_rois[r1] == r2[0])[0]
                    lh_roi_correlation.append(lh_correlation[lh_roi_idx])
                    rh_roi_correlation.append(rh_correlation[rh_roi_idx])
        self.roi_names.append('All vertices')
        lh_roi_correlation.append(lh_correlation)
        rh_roi_correlation.append(rh_correlation)
        lh_median_roi_correlation = [np.median(lh_roi_correlation[r])
            for r in range(len(lh_roi_correlation))]
        rh_median_roi_correlation = [np.median(rh_roi_correlation[r])
            for r in range(len(rh_roi_correlation))]
        return lh_median_roi_correlation, rh_median_roi_correlation
    
class KFold:
    """Run a k-fold cross validation with a given procedure."""
    def __init__(self, folds: int = 8, seed: int = 5, procedure: KFoldProcedure = None) -> None:
        assert folds > 1, "folds must be greater than 1"
        assert seed > 0, "seed must be greater than 0"
        assert isinstance(folds, int), "folds must be an integer"
        assert isinstance(seed, int), "seed must be an integer"
        #assert isinstance(procedure, KFoldProcedure), "procedure must be an instance of KFoldProcedure"
        self.folds = folds
        self.seed = seed
        self.procedure = procedure
        self.fold_correlations = {}
        self.mean_correlations = None

    def run(self) -> None:
        """Runs the procedure on each fold and accesses the correlations."""
        self.procedure.prepare()
        # Create k folds   
        fold_idxs = self.procedure.return_idxs()
        np.random.seed(self.seed)
        np.random.shuffle(fold_idxs)
        self.fold_idxs = np.array_split(fold_idxs, self.folds)

        for fold in range(self.folds):
            # Select validation and train set
            val_idxs = self.fold_idxs[fold]
            train_idxs = np.concatenate([self.fold_idxs[j] for j in range(self.folds) if j != fold])
            
            # Info for current fold
            print(f"#############################################")
            print(f"# Fold: {fold+1}/ {self.folds}")         
            print(f"# Train size: {len(train_idxs)}")
            print(f"# Validation size: {len(val_idxs)}")
            print(f"#############################################")

            # Run procedure
            self.fold_correlations[fold] = self.procedure.run(train_idxs, val_idxs)
        # Get model name and description
        self.model_name, self.description = self.procedure.return_model_name_and_description()
        self.roi_names = self.procedure.return_roi_names()
        self.calculate_mean_accross_folds()
        self.mean_correlations_to_csv()
    
    def calculate_mean_accross_folds(self):
        """Calculates the mean across folds for each layer"""
        self.mean_correlations = {}
        for layer in self.fold_correlations[0].keys():
            self.mean_correlations[layer] = {}
            for hemi in self.fold_correlations[0][layer].keys():
                self.mean_correlations[layer][hemi] = np.nanmean([self.fold_correlations[fold][layer][hemi] for fold in range(self.folds)], axis=0)
    
    def mean_correlations_to_csv(self) -> None:
        df = pd.DataFrame(columns=["model", "layer", "hemisphere", "roi", "correlation"])
        for layer in self.mean_correlations.keys():
                for hemisphere in self.mean_correlations[layer].keys():
                    for i in range(len(self.roi_names)):
                        df = df.append({"model": self.model_name, "layer": layer, "hemisphere": hemisphere, "roi": self.roi_names[i], "correlation": self.mean_correlations[layer][hemisphere][i]}, ignore_index=True)

        validations = glob.glob(f"validations/validation*")
        if len(validations) == 0:
            # create first validation folder
            folder_name = "validation001"
            os.mkdir(f"validations/{folder_name}")
        else:
            # create next validation folder
            last_validation = sorted(validations)[-1]
            last_validation_number = int(last_validation.split("/")[-1].split("validation")[-1])
            next_validation_number = last_validation_number + 1
            folder_name = f"validation{str(next_validation_number).zfill(3)}"
            os.mkdir(f"validations/{folder_name}")

        # Write text file with model description
        with open(f"validations/{folder_name}/info.txt", "w") as f:
            f.write(self.description)

        # Save dataframe
        df.to_csv(f"validations/{folder_name}/results.csv", index=False)

Additionally we define a function to plot the validation results

In [2]:
def plot_kfold_result(validation = "001"):
    """Plots the validation results from the csv file in the given validaiton folder."""
    folder = f"validations/validation{validation}"
    with open(f"{folder}/info.txt", 'r') as f:
        info = f.read()
    df = pd.read_csv(f"{folder}/results.csv")
    # drop model column
    df = df.drop("model", axis=1)

    # Define color palette and assign colors to layers
    palette = sns.color_palette("colorblind", 14)
    layer_colors = {layer: palette[i] for i, layer in enumerate(df.layer.unique())}

    # Split data into left and right hemispheres
    left_df = df[df['hemisphere'] == 'lh']
    right_df = df[df['hemisphere'] == 'rh']

    # Create bar plots
    fig, axes = plt.subplots(2, 1, figsize=(30, 10))
    fig.suptitle(info)
    plt.subplots_adjust(hspace=0.5)
    # reduce white space before first x tick
    plt.margins(x=0.01)

    bar_width = 0.05
    # Plot left hemisphere data
    for i, layer in enumerate(df.layer.unique()):
        layer_data = left_df[left_df['layer'] == layer]
        x = np.arange(len(layer_data['roi']))
        # center bars around xtick
        axes[0].bar(x - len(df.layer.unique())/2 * 0.05 + i * 0.05, layer_data['correlation'], width=bar_width, label=layer, color=layer_colors[layer])

    axes[0].set_xticks([i for i in range(len(layer_data['roi']))])
    axes[0].set_xticklabels([roi for roi in layer_data['roi']])
    axes[0].set_title('Left Hemisphere')
    axes[0].set_xlabel('ROI')
    axes[0].tick_params(axis='x', labelrotation=45)
    axes[0].set_ylabel('Correlation')
    axes[0].legend(loc='upper center', bbox_to_anchor=(0.5, 1), ncol=5)
    axes[0].set_ylim([0, 1])
    row_colors = [layer_colors[layer] for layer in left_df.groupby('layer').mean().sort_values(by='correlation', ascending=False).index]
    axes[0].table(cellText=left_df.groupby('layer').mean().sort_values(by='correlation', ascending=False).round(3).values, rowLabels=df.groupby('layer').mean().sort_values(by='correlation', ascending=False).index, colLabels=["Mean Correlation"], rowColours=row_colors, bbox = [0.95, 0.5, 0.05, 0.5])


    # Plot right hemisphere data
    for i, layer in enumerate(df.layer.unique()):
        layer_data = right_df[right_df['layer'] == layer]
        x = np.arange(len(layer_data['roi']))
        axes[1].bar(x - len(df.layer.unique())/2 * 0.05 + i * 0.05, layer_data['correlation'], width=bar_width, label=layer, color=layer_colors[layer])

    axes[1].set_xticks([i for i in range(len(layer_data['roi']))])
    axes[1].set_xticklabels([roi for roi in layer_data['roi']])
    axes[1].set_title('Right Hemisphere')
    axes[1].set_xlabel('ROI')
    axes[1].tick_params(axis='x', labelrotation=45)
    axes[1].set_ylabel('Correlation')
    axes[1].legend(loc='upper center', bbox_to_anchor=(0.5, 1), ncol=5)
    axes[1].set_ylim([0, 1])
    row_colors = [layer_colors[layer] for layer in right_df.groupby('layer').mean().sort_values(by='correlation', ascending=False).index]
    axes[1].table(cellText=right_df.groupby('layer').mean().sort_values(by='correlation', ascending=False).round(3).values, rowLabels=df.groupby('layer').mean().sort_values(by='correlation', ascending=False).index, colLabels=["Mean Correlation"], rowColours=row_colors, bbox = [0.95, 0.5, 0.05, 0.5])


    plt.show()

#### SubmissionProcedure & CreateSubmission Classes

The SubmissionProcedure class is used to define a procedure which fits a model to predict the test fmri activity. 
It can be supplied to a CreateSubmission class which executes its run() function on all subjects to generate a submission.

In [10]:
class SubmissionProcedure:
    """Used to create a submission procedure that is executed for each subject in the CreateSubmission class."""

    def run(self):
        raise NotImplementedError

class CreateSubmission:
    """Create a new challenge submission."""
    def __init__(self, 
                 subjects : List[Subject],
                 procedure: SubmissionProcedure):
        self.subjects = subjects
        self.procedure = procedure

    def create_submission_folder(self) -> None:
        # create new submission folder with newest name
        submissions = glob.glob(f"submissions/submission*")
        if len(submissions) == 0:
            # create first submission folder
            self.folder_name = "submission001"
            os.mkdir(f"submissions/{self.folder_name}")
        else:
            # create next submission folder
            last_submission = sorted(submissions)[-1]
            last_submission_number = int(last_submission.split("/")[-1].split("submission")[-1])
            next_submission_number = last_submission_number + 1
            self.folder_name = f"submission{str(next_submission_number).zfill(3)}"
            os.mkdir(f"submissions/{self.folder_name}")
        # Write text file with model description
        with open(f"submissions/{self.folder_name}/info.txt", "w") as f:
            f.write(self.procedure.description)
        # create a folder for each subject
        for subject in self.subjects:
            os.mkdir(f"submissions/{self.folder_name}/{subject.subject}")

    def save_predictions(self, subject: Subject, lh_predictions: np.ndarray, rh_predictions: np.ndarray) -> None:
        """Save predictions for a subject."""
        lh_predictions = lh_predictions.astype(np.float32)
        rh_predictions = rh_predictions.astype(np.float32)
        save_path = f"submissions/{self.folder_name}/{subject.subject}"
        # Save predictions
        np.save(f"{save_path}/lh_pred_test.npy", lh_predictions)
        np.save(f"{save_path}/rh_pred_test.npy", rh_predictions)

    def run(self):
        self.create_submission_folder()
        for subject in self.subjects:
            print(f"############################")
            print(f"# Subject: {subject.subject}")
            print(f"############################")
            lh_predictions, rh_predictions = self.procedure.run(subject)
            self.save_predictions(subject, lh_predictions, rh_predictions)
            print(f"\n")

These are the models we submitted to the algonauts 2023 competition.

In [11]:
class CLIPVisionFinalLayerPCA100LinearRegression(SubmissionProcedure):
    def __init__(self):
        self.description = "CLIP Vision Model, Final Layer, PCA 100, Linear Regression, First test submission."

    def run(self, subject: Subject) -> np.ndarray:
        """Run the model on a subject."""
        # Prepare data
        subject.create_dataloaders(processor=processor, batch_size=300)
        subject.load_neural_data()
        train_img_dataloader = subject.train_img_dataloader
        test_img_dataloader = subject.test_img_dataloader
        lh_fmri = subject.lh_fmri
        rh_fmri = subject.rh_fmri
        del subject # free up memory

        # Prepare feature extractor
        train_feature_extractor = CLIPFeatureExtractor(idxs=[], last_hidden_layer=True, model=vis_model, dataloader=train_img_dataloader)
        test_feature_extractor = CLIPFeatureExtractor(idxs=[], last_hidden_layer=True, model=vis_model, dataloader=test_img_dataloader)

        # Extract features
        train_feature_extractor.extract_raw_features_from_model()
        raw_train_features = train_feature_extractor.feature_dict["Final Layer"]
        del train_feature_extractor

        # Fit PCA
        pca = PCA(n_components=100)
        pca_transformed_train_features = pca.fit_transform(torch.tensor(raw_train_features).flatten(1).numpy())
        del raw_train_features

        # Fit linear regression
        lh_lin_reg = LinearRegression().fit(pca_transformed_train_features, lh_fmri)
        rh_lin_reg = LinearRegression().fit(pca_transformed_train_features, rh_fmri)
        del pca_transformed_train_features

        # Extract test features
        test_feature_extractor.extract_raw_features_from_model()
        raw_test_features = test_feature_extractor.feature_dict["Final Layer"]
        del test_feature_extractor

        # Transform test features
        pca_transformed_test_features = pca.transform(torch.tensor(raw_test_features).flatten(1).numpy())
        del raw_test_features
        
        # Predict
        lh_predictions = lh_lin_reg.predict(pca_transformed_test_features)
        rh_predictions = rh_lin_reg.predict(pca_transformed_test_features)
        return lh_predictions, rh_predictions

class CLIPTextFinalLayerPCA100LinearRegression(SubmissionProcedure):
    def __init__(self):
        self.description = "CLIP Text Model, Final Layer, PCA 100, Linear Regression, Second test submission."

    def run(self, subject: Subject) -> np.ndarray:
        """Run the model on a subject."""
        # Prepare data
        subject.create_dataloaders(processor=processor, batch_size=300)
        subject.load_neural_data()
        train_txt_dataloader = subject.train_txt_dataloader
        test_txt_dataloader = subject.test_txt_dataloader
        lh_fmri = subject.lh_fmri
        rh_fmri = subject.rh_fmri
        del subject # free up memory

        # Prepare feature extractor
        train_feature_extractor = CLIPFeatureExtractor(idxs=[], last_hidden_layer=True, model=txt_model, dataloader=train_txt_dataloader)
        test_feature_extractor = CLIPFeatureExtractor(idxs=[], last_hidden_layer=True, model=txt_model, dataloader=test_txt_dataloader)

        # Extract features
        train_feature_extractor.extract_raw_features_from_model()
        raw_train_features = train_feature_extractor.feature_dict["Final Layer"]
        del train_feature_extractor

        # Fit PCA
        pca = PCA(n_components=100)
        pca_transformed_train_features = pca.fit_transform(torch.tensor(raw_train_features).flatten(1).numpy())
        del raw_train_features

        # Fit linear regression
        lh_lin_reg = LinearRegression().fit(pca_transformed_train_features, lh_fmri)
        rh_lin_reg = LinearRegression().fit(pca_transformed_train_features, rh_fmri)
        del pca_transformed_train_features

        # Extract test features
        test_feature_extractor.extract_raw_features_from_model()
        raw_test_features = test_feature_extractor.feature_dict["Final Layer"]
        del test_feature_extractor

        # Transform test features
        pca_transformed_test_features = pca.transform(torch.tensor(raw_test_features).flatten(1).numpy())
        del raw_test_features
        
        # Predict
        lh_predictions = lh_lin_reg.predict(pca_transformed_test_features)
        rh_predictions = rh_lin_reg.predict(pca_transformed_test_features)
        return lh_predictions, rh_predictions


class CLIPVisionLayer7PCA200LinearRegression(SubmissionProcedure):
    def __init__(self):
        self.description = "CLIP Vision Model, Transformer Layer 7, PCA 200, Linear Regression, Determined by 8-fold cross validation against all other layers."

    def run(self, subject: Subject) -> np.ndarray:
        """Run the model on a subject."""
        # Prepare data
        subject.create_dataloaders(processor=processor, batch_size=300)
        subject.load_neural_data()
        train_img_dataloader = subject.train_img_dataloader
        test_img_dataloader = subject.test_img_dataloader
        lh_fmri = subject.lh_fmri
        rh_fmri = subject.rh_fmri
        del subject # free up memory

        # Prepare feature extractor
        train_feature_extractor = CLIPFeatureExtractor(idxs=[7], last_hidden_layer=False, model=vis_model, dataloader=train_img_dataloader)
        test_feature_extractor = CLIPFeatureExtractor(idxs=[7], last_hidden_layer=False, model=vis_model, dataloader=test_img_dataloader)

        # Extract features
        train_feature_extractor.extract_raw_features_from_model()
        raw_train_features = train_feature_extractor.feature_dict["Transformer Layer 7"]
        del train_feature_extractor

        # Fit PCA
        pca = PCA(n_components=200)
        pca_transformed_train_features = pca.fit_transform(torch.tensor(raw_train_features).flatten(1).numpy())
        del raw_train_features

        # Fit linear regression
        lh_lin_reg = LinearRegression().fit(pca_transformed_train_features, lh_fmri)
        rh_lin_reg = LinearRegression().fit(pca_transformed_train_features, rh_fmri)
        del pca_transformed_train_features

        # Extract test features
        test_feature_extractor.extract_raw_features_from_model()
        raw_test_features = test_feature_extractor.feature_dict["Transformer Layer 7"]
        del test_feature_extractor

        # Transform test features
        pca_transformed_test_features = pca.transform(torch.tensor(raw_test_features).flatten(1).numpy())
        del raw_test_features
        
        # Predict
        lh_predictions = lh_lin_reg.predict(pca_transformed_test_features)
        rh_predictions = rh_lin_reg.predict(pca_transformed_test_features)
        return lh_predictions, rh_predictions
    
class CLIPCombinedLayer7PCA200LinearRegression(SubmissionProcedure):
    def __init__(self):
        self.description = "CLIP Vision + Text Model, Transformer Layer 7, PCA 200, Linear Regression, Determined by 8-fold cross validation against all other layers."

    def run(self, subject: Subject) -> np.ndarray:
        """Run the model on a subject."""
        # Prepare data
        subject.create_dataloaders(processor=processor, batch_size=300)
        subject.load_neural_data()
        train_img_dataloader = subject.train_img_dataloader
        test_img_dataloader = subject.test_img_dataloader
        train_txt_dataloader = subject.train_txt_dataloader
        test_txt_dataloader = subject.test_txt_dataloader
        lh_fmri = subject.lh_fmri
        rh_fmri = subject.rh_fmri
        del subject # free up memory

        # Prepare feature extractor
        train_txt_feature_extractor = CLIPFeatureExtractor(idxs=[7], last_hidden_layer=False, model=txt_model, dataloader=train_txt_dataloader)
        test_txt_feature_extractor = CLIPFeatureExtractor(idxs=[7], last_hidden_layer=False, model=txt_model, dataloader=test_txt_dataloader)
        train_img_feature_extractor = CLIPFeatureExtractor(idxs=[7], last_hidden_layer=False, model=vis_model, dataloader=train_img_dataloader)
        test_img_feature_extractor = CLIPFeatureExtractor(idxs=[7], last_hidden_layer=False, model=vis_model, dataloader=test_img_dataloader)

        # Text features
        # Extract features
        train_txt_feature_extractor.extract_raw_features_from_model()
        raw_train_txt_features = train_txt_feature_extractor.feature_dict["Transformer Layer 7"]
        del train_txt_feature_extractor

        # Fit PCA
        txt_pca = PCA(n_components=200)
        pca_transformed_train_txt_features = txt_pca.fit_transform(torch.tensor(raw_train_txt_features).flatten(1).numpy())
        del raw_train_txt_features

        # Image features
        # Extract features
        train_img_feature_extractor.extract_raw_features_from_model()
        raw_train_img_features = train_img_feature_extractor.feature_dict["Transformer Layer 7"]
        del train_img_feature_extractor

        # Fit PCA
        img_pca = PCA(n_components=200)
        pca_transformed_train_img_features = img_pca.fit_transform(torch.tensor(raw_train_img_features).flatten(1).numpy())
        del raw_train_img_features

        # Fit linear regression
        lh_lin_reg = LinearRegression().fit(np.hstack([pca_transformed_train_txt_features, pca_transformed_train_img_features]), lh_fmri)
        rh_lin_reg = LinearRegression().fit(np.hstack([pca_transformed_train_txt_features, pca_transformed_train_img_features]), rh_fmri)
        del pca_transformed_train_txt_features, pca_transformed_train_img_features

        # Text features
        # Extract test features
        test_txt_feature_extractor.extract_raw_features_from_model()
        raw_test_txt_features = test_txt_feature_extractor.feature_dict["Transformer Layer 7"]
        del test_txt_feature_extractor

        # Transform test features
        pca_transformed_test_txt_features = txt_pca.transform(torch.tensor(raw_test_txt_features).flatten(1).numpy())
        del raw_test_txt_features

        # Image features
        # Extract test features
        test_img_feature_extractor.extract_raw_features_from_model()
        raw_test_img_features = test_img_feature_extractor.feature_dict["Transformer Layer 7"]
        del test_img_feature_extractor

        # Transform test features
        pca_transformed_test_img_features = img_pca.transform(torch.tensor(raw_test_img_features).flatten(1).numpy())
        del raw_test_img_features   
        
        # Predict
        lh_predictions = lh_lin_reg.predict(np.hstack([pca_transformed_test_txt_features, pca_transformed_test_img_features]))
        rh_predictions = rh_lin_reg.predict(np.hstack([pca_transformed_test_txt_features, pca_transformed_test_img_features]))
        return lh_predictions, rh_predictions

## Cross Validations

We ran three 8-fold cross validation procedures on a vision only model, text only model and combined model; all on data of subject 1.



### CLIP Vision Model, PCA200, Linear Regression <a id='cv_vis_pca200_linreg'></a>

We ran 8 folds on image data of subject 1. For each iteration the current fold was selected as the validation set and the remaining folds were used as the training set.

The validation procedure for each fold was as follows: 

- Supplying the training set to a CLIPVisionModel.
- Extracting the raw features for all layers (Embedding, Transformer 1 - 12, Final Layer).
- Looping over folds:
    - Assing training and validation sets for images and fmri activity.
    - Looping over each layer:
        - Fitting a PCA with 200 components using the raw training features.
        - Transforming the raw training features using the fitted PCA.
        - Fitting a linear regression, predicting training fmri activity from the pca transformed training features. (for left and right hemisphere)
        - Transforming the raw validation features using the fitted PCA.
        - Using the fitted linear models to predict validation fmri activity from the pca transformed validation features. (for left and right hemisphere)
        - Calculating correlations between predicted and actual fmri activity for the validation set. (for left and right hemisphere)
        - Calculating median correlations for each of the 36 challenge ROI. (for left and right hemisphere)
    - Storing the median correlations for each hemisphere and each layer for the current fold.
- Calculating mean median correlations accross all folds for each layer. (for left and right hemisphere)
- Plotting the mean median correlations for each layer for each ROI and select the layer with the highest mean median correlation averaged over all ROIs.

Transformer layer 7 achieved the highest overall performance (left hemisphere = 0.472, right hemisphere = 0.484) and was selected for submission (submission score = 49.3178).
    


Interestingly the plot below suggests, that earlier layers are better at predicting the activity of the earlier visual cortex (V1, V2, V3, V4) and later layers are better at predicting the activity of the other ROIs.

<img src="plots/8-fold cross validation vision model pca 200 all layers.png" alt="Alternative text" />

#### KFoldProcedure Class <a id='kfpc_single_clip_single_subject'></a>

In [ ]:
class KFoldSingleCLIPSingleSubject(KFoldProcedure):
    """A procedure that runs k-fold on all layers of a single CLIP model on a single subject."""
    def __init__(self, 
                 feature_extractor: CLIPFeatureExtractor,
                 subject: Subject, 
                 pca: PCA,
                 model_name: str = None,
                 description: str = None) -> None:
        assert isinstance(feature_extractor, CLIPFeatureExtractor), "feature_extractor must be an instance of CLIPFeatureExtractor"
        assert isinstance(subject, Subject), "subject must be an instance of Subject"
        self.feature_extractor = feature_extractor
        self.subject = subject
        self.pca = pca
        self.model_name = model_name
        self.description = description
        self.correlations = {}
        super().__init__()

    def prepare(self):
        # Extract raw features
        self.feature_extractor.extract_raw_features_from_model()
        self.raw_features = self.feature_extractor.feature_dict
        self.feature_extractor = None # free memory

        # Load challenge rois
        self.subject.load_challenge_rois()
        self.lh_challenge_rois = self.subject.lh_challenge_rois
        self.rh_challenge_rois = self.subject.rh_challenge_rois
        self.roi_name_maps = self.subject.roi_name_maps

        # Load neural data
        self.subject.load_neural_data()
        self.lh_fmri = self.subject.lh_fmri
        self.rh_fmri = self.subject.rh_fmri
        self.subject = None # free memory

        # Prepare correlation dict
        self.fold_correlations = {}

    def run(self, train_idxs: np.ndarray, val_idxs: np.ndarray) -> Dict[str, Dict[str, np.ndarray]]:
        # Loop over all layers          
        correlations = {}
        for layer in self.raw_features.keys():
            print(f"> {layer}")
            # Assign train and val features
            train_features = self.raw_features[layer][train_idxs]
            val_features = self.raw_features[layer][val_idxs]

            # Assign train and val fmri
            train_lh_fmri = self.lh_fmri[train_idxs]
            train_rh_fmri = self.rh_fmri[train_idxs]
            val_lh_fmri = self.lh_fmri[val_idxs]
            val_rh_fmri = self.rh_fmri[val_idxs]

            # Fit PCA models
            print(f"Fitting PCA model for {layer}...")
            train_pca_features = self.pca.fit_transform(torch.tensor(train_features).flatten(1).numpy())
            del train_features # free memory

            # Fit linear regression
            print(f"Fitting linear regression models for {layer}...")
            lh_lin_reg = LinearRegression().fit(train_pca_features, train_lh_fmri)
            rh_lin_reg = LinearRegression().fit(train_pca_features, train_rh_fmri)
            del train_pca_features, train_lh_fmri, train_rh_fmri # free memory

            # Transform validation features
            print(f"Transforming validation features for {layer}...")
            val_txt_pca_features = self.pca.transform(torch.tensor(val_features).flatten(1).numpy())
            del val_features # free memory

            # Predict validation set
            print(f"Predicting validation set for {layer}...")
            lh_val_pred = lh_lin_reg.predict(val_txt_pca_features)
            rh_val_pred = rh_lin_reg.predict(val_txt_pca_features)
            del val_txt_pca_features, lh_lin_reg, rh_lin_reg # free memory
            
            # Calculate correlations
            print(f"Calculating correlations for {layer}...\n")
            lh_correlation, rh_correlation = self.calculate_correlations(lh_val_pred, rh_val_pred, val_lh_fmri, val_rh_fmri)
            lh_median_roi_correlation, rh_median_roi_correlation = self.calculate_median_correlations(lh_correlation, rh_correlation)
            
            # Store correlations
            correlations[layer] = {"lh": lh_median_roi_correlation, "rh": rh_median_roi_correlation} 
        return correlations

    def return_idxs(self) -> np.ndarray:
        return np.arange(len(self.raw_features[list(self.raw_features.keys())[0]])) 
    

#### Code

In [ ]:
# Raise exception to prevent accidental execution
raise Exception("This code is not meant to be run. It is only meant to be used as a reference for the code used to generate the submission.")

# Initializing all required objects
subject = Subject("subj01")
subject.create_dataloaders(processor=processor, batch_size=300)
feature_extractor = CLIPFeatureExtractor(
    idxs=[0,1,2,3,4,5,6,7,8,9,10,11,12], 
    last_hidden_layer=True, 
    model=vis_model, # here we use the vis_model
    dataloader=subject.train_img_dataloader) # and the img_dataloader

# Initialize the kfold procedure
kfold_procedure = KFoldSingleCLIPSingleSubject(
    feature_extractor=feature_extractor,
    subject=subject,
    pca=PCA(n_components=200),
    model_name="CLIP Vision",
    description="CLIP Vision Model, all layers, PCA 200, Single layer linear regression, 8-fold cross validation"
)

# Run KFold
KFold(folds=8, seed=5, procedure=kfold_procedure).run()

#### Additional Data Analysis

Because The cross validation plot indicated that earlier layers are better at predicting earlier ROIs, we checked which layer would have the highest mean median correlation averaged over early ROI (V1, V2, V3) and which layer has the highest mean median correlation averaged over all other ROI. 

The results indicate that Transformer Layer 4 is best at predicting activity in the early visual cortex, while layer 8 is the best layer of the rest of the cortex. 
Because of these findings we decided cross validate another model agains our strongest model so far. 

In [111]:
validation_folder = "validations"
validation = "001"

df = pd.read_csv(f"{validation_folder}/validation{validation}/results.csv")
df = df.drop(columns="model")

early_roi = ["V1v", "V1d", "V2v", "V2d", "V3v", "V3d"]
best_early_lh = df[(df.roi.isin(early_roi)) & (df.hemisphere == "lh")].groupby(["layer"]).mean(["correlations"]).sort_values(by='correlation', ascending=False).round(3).head(1).index.values[0]
best_early_lh_cor = df[(df.roi.isin(early_roi)) & (df.hemisphere == "lh")].groupby(["layer"]).mean(["correlations"]).sort_values(by='correlation', ascending=False).round(3).head(1).values[0][0]
best_early_rh = df[(df.roi.isin(early_roi)) & (df.hemisphere == "rh")].groupby(["layer"]).mean(["correlations"]).sort_values(by='correlation', ascending=False).round(3).head(1).index.values[0]
best_early_rh_cor = df[(df.roi.isin(early_roi)) & (df.hemisphere == "rh")].groupby(["layer"]).mean(["correlations"]).sort_values(by='correlation', ascending=False).round(3).head(1).values[0][0]
best_late_lh = df[~(df.roi.isin(early_roi)) & (df.hemisphere == "lh")].groupby(["layer"]).mean(["correlations"]).sort_values(by='correlation', ascending=False).round(3).head(1).index.values[0]
best_late_lh_cor = df[~(df.roi.isin(early_roi)) & (df.hemisphere == "lh")].groupby(["layer"]).mean(["correlations"]).sort_values(by='correlation', ascending=False).round(3).head(1).values[0][0]
best_late_rh = df[~(df.roi.isin(early_roi)) & (df.hemisphere == "rh")].groupby(["layer"]).mean(["correlations"]).sort_values(by='correlation', ascending=False).round(3).head(1).index.values[0]
best_late_rh_cor = df[~(df.roi.isin(early_roi)) & (df.hemisphere == "rh")].groupby(["layer"]).mean(["correlations"]).sort_values(by='correlation', ascending=False).round(3).head(1).values[0][0]
print(f"The best layer for early ROI on the left hemisphere is\t\t==>\t{best_early_lh} ({best_early_lh_cor})")
print(f"The best layer for early ROI on the right hemisphere is\t\t==>\t{best_early_rh} ({best_early_rh_cor})")
print(f"The best layer for later ROI on the left hemisphere is\t\t==>\t{best_late_lh} ({best_late_lh_cor})")
print(f"The best layer for later ROI on the right hemisphere is\t\t==>\t{best_late_rh} ({best_late_rh_cor})")


The best layer for early ROI on the left hemisphere is		==>	Transformer Layer 4 (0.566)
The best layer for early ROI on the right hemisphere is		==>	Transformer Layer 4 (0.568)
The best layer for later ROI on the left hemisphere is		==>	Transformer Layer 8 (0.458)
The best layer for later ROI on the right hemisphere is		==>	Transformer Layer 8 (0.473)


### CLIP Text Model, PCA200, Linear Regression <a id='cv_txt_pca200_linreg'></a>

We ran 8 folds on caption data of subject 1. For each iteration the current fold was selected as the validation set and the remaining folds were used as the training set.

The validation procedure for each fold was as follows: 

- Supplying the training set to a CLIPTextModel.
- Extracting the raw features for all layers (Embedding, Transformer 1 - 12, Final Layer).
- Looping over folds:
    - Assing training and validation sets for coco captions and fmri activity.
    - Looping over each layer:
        - Fitting a PCA with 200 components using the raw training features.
        - Transforming the raw training features using the fitted PCA.
        - Fitting a linear regression, predicting training fmri activity from the pca transformed training features. (for left and right hemisphere)
        - Transforming the raw validation features using the fitted PCA.
        - Using the fitted linear models to predict validation fmri activity from the pca transformed validation features. (for left and right hemisphere)
        - Calculating correlations between predicted and actual fmri activity for the validation set. (for left and right hemisphere)
        - Calculating median correlations for each of the 36 challenge ROI. (for left and right hemisphere)
    - Storing the median correlations for each hemisphere and each layer for the current fold.
- Calculating mean median correlations accross all folds for each layer. (for left and right hemisphere)
- Plotting the mean median correlations for each layer for each ROI and select the layer with the highest mean median correlation averaged over all ROIs.

Transformer layer 11 achieved the highest overall performance (left hemisphere = 0.293, right hemisphere = 0.306) but was much worse than transformer layer 7 of the vision only model (left hemisphere = 0.472, right hemisphere = 0.484), therefore we didnt submit this.
    


As opposed to the vision only model, the plot below indicates that later levels of the text only model are better to predict activity in all ROIs. Additionally, the text only model performed much worse on the early visual cortex (V1, V2, V3, V4) than the vision only model but performed comparably for the later layers.

<img src="plots/8-fold cross validation text model pca 200 all layers.png" alt="Alternative text" />

#### KFoldProcedure Class ([Same as for the Vision Model](#kfpc_single_clip_single_subject))

#### Code

In [12]:
# Raise exception to prevent accidental execution
raise Exception("This code is not meant to be run. It is only meant to be used as a reference for the code used to generate the submission.")

# Initializing all required objects
subject = Subject("subj01")
subject.create_dataloaders(processor=processor, batch_size=300)
feature_extractor = CLIPFeatureExtractor(
    idxs=[0,1,2,3,4,5,6,7,8,9,10,11,12], 
    last_hidden_layer=True, 
    model=txt_model, # here we use the txt_model 
    dataloader=subject.train_txt_dataloader) # and the txt_dataloader

# Initialize the kfold procedure
kfold_procedure = KFoldSingleCLIPSingleSubject(
    feature_extractor=feature_extractor,
    subject=subject,
    pca=PCA(n_components=200),
    model_name="CLIP Text",
    description="CLIP Text Model, all layers, PCA 200, Single layer linear regression, 8-fold cross validation"
)

# Run KFold
KFold(folds=8, seed=5, procedure=kfold_procedure).run()

### CLIP Text + Vision Model, PCA200, Linear Regression <a id='cv_txtvis_pca200_linreg'></a>

We ran 8 folds on caption and image data of subject 1. For each iteration the current fold was selected as the validation set and the remaining folds were used as the training set.
Because of RAM limitations we ran the cross validation for the combined model in three batches (Embedding Layer to Transformer Layer 4, Transformer Layers 5 to 9, Transformer Layer 10 to Final Layer). Afterwards we manually combined the csv files that contained the results to provide one plot including all layers.

The validation procedure for each fold was as follows: 

- Supplying the caption training set to a CLIPTextModel and the image training set to a CLIPVisionModel.
- Extracting the raw features for all layers (Embedding, Transformer 1 - 12, Final Layer). (for text and images)
- Looping over folds:
    - Assing training and validation sets for images, coco captions and fmri activity.
    - Looping over each layer:
        - Fitting a PCA with 200 components using the raw training features. (for text and images)
        - Transforming the raw training features using the fitted PCA. (for text and images)
        - Fitting a linear regression, predicting training fmri activity from the combination of pca transformed image and caption training features. (for left and right hemisphere)
        - Transforming the raw validation features using the fitted PCA. (for text and images)
        - Using the fitted linear models to predict validation fmri activity from the combination of pca transformed image and caption validation features. (for left and right hemisphere)
        - Calculating correlations between predicted and actual fmri activity for the validation set. (for left and right hemisphere)
        - Calculating median correlations for each of the 36 challenge ROI. (for left and right hemisphere)
    - Storing the median correlations for each hemisphere and each layer for the current fold.
- Calculating mean median correlations accross all folds for each layer. (for left and right hemisphere)
- Plotting the mean median correlations for each layer for each ROI and select the layer with the highest mean median correlation averaged over all ROIs.

Similar to the vision only model, transformer layer 7 achieved the highest overall performance (left hemisphere = 0.468, right hemisphere = 0.48) and was selected for submission (submission score = NA).
    


<img src="plots/8-fold cross validation combined model pca 200 all layers.png" alt="Alternative text" />

#### KFoldProcedure Class

In [35]:
class KFoldCombinedCLIPSingleSubject(KFoldProcedure):
    """A procedure that runs k-fold on all layers of a combination of the text and vision CLIP model on a single subject."""
    def __init__(self, 
                 subject: Subject, 
                 txt_pca: PCA,
                 img_pca: PCA,
                 model_name: str = None,
                 description: str = None,
                 layers: list = [],
                 last_hidden_state = False) -> None:
        self.subject = subject
        self.txt_pca = txt_pca
        self.img_pca = img_pca
        self.model_name = model_name
        self.description = description
        self.layers = layers.copy()
        self.layers2 = layers.copy()
        self.last_hidden_state = last_hidden_state
        self.correlations = {}
        super().__init__()

    def prepare(self):
        # Prepare data
        self.subject.create_dataloaders(processor, batch_size=400)
        train_img_dataloader = self.subject.train_img_dataloader
        train_txt_dataloader = self.subject.train_txt_dataloader

        # Extract raw features from text model
        feature_extractor = CLIPFeatureExtractor(idxs=self.layers, last_hidden_layer=self.last_hidden_state, model=txt_model, dataloader=train_txt_dataloader)
        feature_extractor.extract_raw_features_from_model()
        self.raw_txt_features = feature_extractor.feature_dict
        del feature_extractor # free memory

        # Extract raw features from vision model
        feature_extractor = CLIPFeatureExtractor(idxs=self.layers2, last_hidden_layer=self.last_hidden_state, model=vis_model, dataloader=train_img_dataloader)
        feature_extractor.extract_raw_features_from_model()
        self.raw_img_features = feature_extractor.feature_dict
        del feature_extractor # free memory

        # Load challenge rois
        self.subject.load_challenge_rois()
        self.lh_challenge_rois = self.subject.lh_challenge_rois
        self.rh_challenge_rois = self.subject.rh_challenge_rois
        self.roi_name_maps = self.subject.roi_name_maps

        # Load neural data
        self.subject.load_neural_data()
        self.lh_fmri = self.subject.lh_fmri
        self.rh_fmri = self.subject.rh_fmri
        self.subject = None # free memory

        # Prepare correlation dict
        self.fold_correlations = {}

    def run(self, train_idxs: np.ndarray, val_idxs: np.ndarray) -> Dict[str, Dict[str, np.ndarray]]:
        # Loop over all layers          
        correlations = {}
        for layer in self.raw_txt_features.keys():
            print(f"> {layer}")
            # Assign train and val text features
            train_txt_features = self.raw_txt_features[layer][train_idxs]
            val_txt_features = self.raw_txt_features[layer][val_idxs]
            # Assign train and val vision features
            train_img_features = self.raw_img_features[layer][train_idxs]
            val_img_features = self.raw_img_features[layer][val_idxs]

            # Assign train and val fmri
            train_lh_fmri = self.lh_fmri[train_idxs]
            train_rh_fmri = self.rh_fmri[train_idxs]
            val_lh_fmri = self.lh_fmri[val_idxs]
            val_rh_fmri = self.rh_fmri[val_idxs]

            # Fit PCA models
            print(f"Fitting PCA model for {layer}...")
            train_txt_pca_features = self.txt_pca.fit_transform(torch.tensor(train_txt_features).flatten(1).numpy())
            del train_txt_features # free memory
            train_img_pca_features = self.img_pca.fit_transform(torch.tensor(train_img_features).flatten(1).numpy())
            del train_img_features # free memory

            # Fit linear regression
            print(f"Fitting linear regression models for {layer}...")
            lh_lin_reg = LinearRegression().fit(np.hstack([train_txt_pca_features, train_img_pca_features]), train_lh_fmri)
            rh_lin_reg = LinearRegression().fit(np.hstack([train_txt_pca_features, train_img_pca_features]), train_rh_fmri)
            del train_txt_pca_features, train_img_pca_features, train_lh_fmri, train_rh_fmri # free memory

            # Transform validation features
            print(f"Transforming validation features for {layer}...")
            val_txt_pca_features = self.txt_pca.transform(torch.tensor(val_txt_features).flatten(1).numpy())
            del val_txt_features # free memory
            val_img_pca_features = self.img_pca.transform(torch.tensor(val_img_features).flatten(1).numpy())
            del val_img_features # free memory

            # Predict validation set
            print(f"Predicting validation set for {layer}...")
            lh_val_pred = lh_lin_reg.predict(np.hstack([val_txt_pca_features, val_img_pca_features]))
            rh_val_pred = rh_lin_reg.predict(np.hstack([val_txt_pca_features, val_img_pca_features]))
            del val_txt_pca_features, val_img_pca_features, lh_lin_reg, rh_lin_reg # free memory
            
            # Calculate correlations
            print(f"Calculating correlations for {layer}...\n")
            lh_correlation, rh_correlation = self.calculate_correlations(lh_val_pred, rh_val_pred, val_lh_fmri, val_rh_fmri)
            lh_median_roi_correlation, rh_median_roi_correlation = self.calculate_median_correlations(lh_correlation, rh_correlation)
            
            # Store correlations
            correlations[layer] = {"lh": lh_median_roi_correlation, "rh": rh_median_roi_correlation} 
        return correlations

    def return_idxs(self) -> np.ndarray:
        return np.arange(len(self.raw_txt_features[list(self.raw_txt_features.keys())[0]])) 

#### Code

In [ ]:
# Raise exception to prevent accidental execution
raise Exception("This code is not meant to be run. It is only meant to be used as a reference for the code used to generate the submission.")

# Initialize required objects
subject = Subject("subj01")

# Initialize kfold procedure
kfold_procedure = KFoldCombinedCLIPSingleSubject(
    subject=subject, 
    txt_pca=PCA(n_components=200), 
    img_pca=PCA(n_components=200), 
    model_name="CLIP Vision + Text", 
    description="CLIP Vision + Text Model, Embedding layer and first 4 Transformer Layers, PCA 200, Single layer linear regression, 8-fold cross validation", 
    layers=[0,1,2,3,4], # only first 5 layers
    last_hidden_state=False
    )

# Run first kfold
KFold(folds=8, procedure=kfold_procedure).run()

# Initalize kfold procedure for second batch
kfold_procedure = KFoldCombinedCLIPSingleSubject(
    subject=subject, 
    txt_pca=PCA(n_components=200), 
    img_pca=PCA(n_components=200), 
    model_name="CLIP Vision + Text", 
    description="CLIP Vision + Text Model, Transformer layers 5 to 9, PCA 200, Single layer linear regression, 8-fold cross validation", 
    layers=[5,6,7,8,9], # the next 5 layers
    last_hidden_state=False
    )
# Run second kfold
KFold(folds=8, procedure=kfold_procedure).run()

# Initalize kfold procedure for third batch
kfold_procedure = KFoldCombinedCLIPSingleSubject(
    subject=subject, 
    txt_pca=PCA(n_components=200), 
    img_pca=PCA(n_components=200), 
    model_name="CLIP Vision + Text", 
    description="CLIP Vision + Text Model, Transformer layers 10 to 12 and Final Layer, PCA 200, Single layer linear regression, 8-fold cross validation", 
    layers=[10,11,12], # the next 4 layers
    last_hidden_state=True # and final layer
    )
# Run third kfold
KFold(folds=8, procedure=kfold_procedure).run()

In [36]:
######################### Still need to run this! ###########################
subject = Subject("subj01")

# Initalize kfold procedure for third batch
kfold_procedure = KFoldCombinedCLIPSingleSubject(
    subject=subject, 
    txt_pca=PCA(n_components=200), 
    img_pca=PCA(n_components=200), 
    model_name="CLIP Vision + Text", 
    description="CLIP Vision + Text Model, Transformer layers 10 to 12 and Final Layer, PCA 200, Single layer linear regression, 8-fold cross validation", 
    layers=[10,11,12], # the next 4 layers
    last_hidden_state=True # and final layer
    )

# Run second kfold
KFold(folds=8, procedure=kfold_procedure).run()

Training images: 9841
Test images: 159
Training captions: 9841
Test captions: 159
Train image dataloader: 25 batches
Test image dataloader: 1 batches
Train caption dataloader: 25 batches
Test caption dataloader: 1 batches
idxs: [10, 11, 12, 13]
feature dict keys: dict_keys(['Transformer Layer 10', 'Transformer Layer 11', 'Transformer Layer 12', 'Final Layer'])


100%|██████████| 25/25 [00:41<00:00,  1.66s/it]


idxs: [10, 11, 12, 13]
feature dict keys: dict_keys(['Transformer Layer 10', 'Transformer Layer 11', 'Transformer Layer 12', 'Final Layer'])


100%|██████████| 25/25 [02:33<00:00,  6.13s/it]


Left hemisphere neural data loaded. Shape: (9841, 19004)
Right hemisphere neural data loaded. Shape: (9841, 20544)
#############################################
# Fold: 1/ 8
# Train size: 8610
# Validation size: 1231
#############################################
> Transformer Layer 10
Fitting PCA model for Transformer Layer 10...
Fitting linear regression models for Transformer Layer 10...
Transforming validation features for Transformer Layer 10...
Predicting validation set for Transformer Layer 10...
Calculating correlations for Transformer Layer 10...



...

100%|██████████| 20544/20544 [00:18<00:00, 1094.89it/s]
c:\Users\luke-\Desktop\Python Repositories\algonauts-2023\.conda\lib\site-packages\numpy\core\fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
c:\Users\luke-\Desktop\Python Repositories\algonauts-2023\.conda\lib\site-packages\numpy\core\_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


> Transformer Layer 12
Fitting PCA model for Transformer Layer 12...
Fitting linear regression models for Transformer Layer 12...
Transforming validation features for Transformer Layer 12...
Predicting validation set for Transformer Layer 12...
Calculating correlations for Transformer Layer 12...



100%|██████████| 20544/20544 [00:18<00:00, 1105.22it/s]
c:\Users\luke-\Desktop\Python Repositories\algonauts-2023\.conda\lib\site-packages\numpy\core\fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
c:\Users\luke-\Desktop\Python Repositories\algonauts-2023\.conda\lib\site-packages\numpy\core\_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


> Final Layer
Fitting PCA model for Final Layer...
Fitting linear regression models for Final Layer...
Transforming validation features for Final Layer...
Predicting validation set for Final Layer...
Calculating correlations for Final Layer...



100%|██████████| 20544/20544 [00:18<00:00, 1093.07it/s]
c:\Users\luke-\Desktop\Python Repositories\algonauts-2023\.conda\lib\site-packages\numpy\core\fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
c:\Users\luke-\Desktop\Python Repositories\algonauts-2023\.conda\lib\site-packages\numpy\core\_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
C:\Users\luke-\AppData\Local\Temp\ipykernel_24516\1260140408.py:112: RuntimeWarning: Mean of empty slice
  self.mean_correlations[layer][hemi] = np.nanmean([self.fold_correlations[fold][layer][hemi] for fold in range(self.folds)], axis=0)
C:\Users\luke-\AppData\Local\Temp\ipykernel_24516\1260140408.py:119: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({"model": self.model_name, "layer": layer, "hemisphere": hemisphere, "roi":

### CLIP Vision Model, Transformer Layer 4 + Transformer Layer 8, PCA200, Linear Regression

Because of our findings in the first cross validation we decided to compare our best Vision Model (PCA200, Layer 7) with a combined Vision Model (PCA200, Layer4 + Layer8). 

We ran 8 folds on image data of subject 1. For each iteration the current fold was selected as the validation set and the remaining folds were used as the training set.

The validation procedure for each fold was as follows: 

Preparation

- Supplying the training set to a CLIPVisionModel.
- Extracting the raw features for Transformer Layers 4, 7 and 8.

Looping over all folds
- For each fold:
    - Assigning training and validation sets for images and fmri activity.
    
    Layer 7
    
    - Fit PCA with 200 components using the raw training features of layer 7.
    - Transforming the raw training features using the fitted PCA.
    - Fitting a linear regression, predicting training fmri activity from the pca transformed training features. (for left and right hemisphere)
    - Transforming the raw validation features using the fitted PCA.
    - Using the fitted linear models to predict validation fmri activity from the pca transformed validation features. (for left and right hemisphere)
    - Calculating correlations between predicted and actual fmri activity for the validation set. (for left and right hemisphere)
    - Calculating median correlations for each of the 36 challenge ROI. (for left and right hemisphere)
    - Storing the median correlations for each hemisphere for layer 7 for the current fold.

    Layers 4 and 8
    
    - Combine raw training features of layer 4 and 8.
    - Fit PCA with 200 components using the combined raw training features of layers 4 and 8.
    - Transforming the raw training features using the fitted PCA.
    - Fitting a linear regression, predicting training fmri activity from the pca transformed training features. (for left and right hemisphere)
    - Transforming the raw validation features using the fitted PCA.
    - Using the fitted linear models to predict validation fmri activity from the pca transformed validation features. (for left and right hemisphere)
    - Calculating correlations between predicted and actual fmri activity for the validation set. (for left and right hemisphere)
    - Calculating median correlations for each of the 36 challenge ROI. (for left and right hemisphere)
    - Storing the median correlations for each hemisphere for the combination of layers 4 and 8 for the current fold.
    
After the validation

- Calculating mean median correlations accross all folds for each layer. (for left and right hemisphere)
- Plotting the mean median correlations for each layer for each ROI and select the layer with the highest mean median correlation averaged over all ROIs.   


#### KFoldProcedure CLass

In [ ]:
class KFoldVisPCA200L7VersusL4L8(KFoldProcedure):
    """A procedure that runs k-fold on all layers of a single CLIP model on a single subject."""
    def __init__(self, 
                 subject: Subject, 
                 pca: PCA,
                 model_name: str = None,
                 description: str = None) -> None:
        assert isinstance(subject, Subject), "subject must be an instance of Subject"
        self.subject = subject
        self.pca = pca
        self.model_name = model_name
        self.description = description
        self.correlations = {}
        super().__init__()

    def prepare(self):
        # Prepare data
        subject.create_dataloaders()
        train_dataloader = subject.train_img_dataloader
        
        # Extract raw features
        feature_extractor = CLIPFeatureExtractor(idxs=[4,7,8], last_hidden_layer=False, model=vis_model, dataloader=train_dataloader)
        feature_extractor.extract_raw_features_from_model()
        self.raw_features = self.feature_extractor.feature_dict
        del feature_extractor # free memory

        # Load challenge rois
        self.subject.load_challenge_rois()
        self.lh_challenge_rois = self.subject.lh_challenge_rois
        self.rh_challenge_rois = self.subject.rh_challenge_rois
        self.roi_name_maps = self.subject.roi_name_maps

        # Load neural data
        self.subject.load_neural_data()
        self.lh_fmri = self.subject.lh_fmri
        self.rh_fmri = self.subject.rh_fmri
        self.subject = None # free memory

        # Prepare correlation dict
        self.fold_correlations = {}

    def run(self, train_idxs: np.ndarray, val_idxs: np.ndarray) -> Dict[str, Dict[str, np.ndarray]]:        
        correlations = {}

        # Assign train and val fmri
        train_lh_fmri = self.lh_fmri[train_idxs]
        train_rh_fmri = self.rh_fmri[train_idxs]
        val_lh_fmri = self.lh_fmri[val_idxs]
        val_rh_fmri = self.rh_fmri[val_idxs]
        
        ####### Layer 7 #######
        # Assign train and val features
        train_features = self.raw_features["Transformer Layer 7"][train_idxs]
        val_features = self.raw_features["Transformer Layer 7"][val_idxs]

        # Fit PCA model
        train_pca_features = self.pca.fit_transform(torch.tensor(train_features).flatten(1).numpy())
        del train_features # free memory

        # Fit linear regressions
        lh_lin_reg = LinearRegression().fit(train_pca_features, train_lh_fmri)
        rh_lin_reg = LinearRegression().fit(train_pca_features, train_rh_fmri)
        del train_pca_features # free memory

        # Transform validation features
        val_txt_pca_features = self.pca.transform(torch.tensor(val_features).flatten(1).numpy())
        del val_features # free memory

        # Predict validation dict
        lh_val_pred = lh_lin_reg.predict(val_txt_pca_features)
        rh_val_pred = rh_lin_reg.predict(val_txt_pca_features)
        del val_txt_pca_features, lh_lin_reg, rh_lin_reg # free memory

        # Calculate correlations
        lh_correlation, rh_correlation = self.calculate_correlations(lh_val_pred, rh_val_pred, val_lh_fmri, val_rh_fmri)
        lh_median_roi_correlation, rh_median_roi_correlation = self.calculate_median_correlations(lh_correlation, rh_correlation)
        
        # Store correlations
        correlations["Transformer Layer 7"] = {"lh": lh_median_roi_correlation, "rh": rh_median_roi_correlation} 

        ####### Layers 4 and 8 #######
        # Assign train and val features
        train_features = self.raw_features["Transformer Layer 4"][train_idxs]
        train_features = np.hstack([train_features, self.raw_features["Transformer Layer 8"][train_idxs]])
        val_features = self.raw_features["Transformer Layer 4"][val_idxs]
        val_features = np.hstack([val_features, self.raw_features["Transformer Layer 8"][val_idxs]])

        # Fit PCA model
        train_pca_features = self.pca.fit_transform(torch.tensor(train_features).flatten(1).numpy())
        del train_features # free memory

        # Fit linear regressions
        lh_lin_reg = LinearRegression().fit(train_pca_features, train_lh_fmri)
        rh_lin_reg = LinearRegression().fit(train_pca_features, train_rh_fmri)
        del train_pca_features # free memory

        # Transform validation features
        val_txt_pca_features = self.pca.transform(torch.tensor(val_features).flatten(1).numpy())
        del val_features # free memory

        # Predict validation dict
        lh_val_pred = lh_lin_reg.predict(val_txt_pca_features)
        rh_val_pred = rh_lin_reg.predict(val_txt_pca_features)
        del val_txt_pca_features, lh_lin_reg, rh_lin_reg # free memory

        # Calculate correlations
        lh_correlation, rh_correlation = self.calculate_correlations(lh_val_pred, rh_val_pred, val_lh_fmri, val_rh_fmri)
        lh_median_roi_correlation, rh_median_roi_correlation = self.calculate_median_correlations(lh_correlation, rh_correlation)
        
        # Store correlations
        correlations["Transformer Layer 4 + 8"] = {"lh": lh_median_roi_correlation, "rh": rh_median_roi_correlation} 
        return correlations

    def return_idxs(self) -> np.ndarray:
        return np.arange(len(self.raw_features[list(self.raw_features.keys())[0]])) 
    

#### Code

In [ ]:
# Raise exception to prevent accidental execution
raise Exception("This code is not meant to be run. It is only meant to be used as a reference for the code used to generate the submission.")

# Initialize required objects
subject = Subject("subj01")

# Initialize kfold procedure
kfold_procedure = KFoldVisPCA200L7VersusL4L8(
    subject=subject, 
    pca=PCA(n_components=200), 
    model_name="CLIP Vision", 
    description="CLIP Vision, Layer 7 versus Layer 4+8, PCA 200, Linear regression, 8-fold cross validation", 
    )

# Run kfold
KFold(folds=8, procedure=kfold_procedure).run()

In [ ]:
################## This still needs to run ###################

# Initialize required objects
subject = Subject("subj01")

# Initialize kfold procedure
kfold_procedure = KFoldVisPCA200L7VersusL4L8(
    subject=subject, 
    pca=PCA(n_components=200), 
    model_name="CLIP Vision", 
    description="CLIP Vision, Layer 7 versus Layer 4+8, PCA 200, Linear regression, 8-fold cross validation", 
    )

# Run kfold
KFold(folds=8, procedure=kfold_procedure).run()


## Challenge Submissions

We wrote two classes CreateSubmission and SubmissionProcedure to make the process of generating predictions for submission as straight forward as possible. 

The CreateSubmission class creates folders for submission and loops over all subjects to apply the specific SubmissionProcedure.

### First and Second Submission - Vision Only Final Layer/ Text Only Final Layer, PCA 100

To familiarize ourselves with the CodaLab submission system we selected our first two models without cross validation and used a 90%-10% train-validation split to guide our decision instead. 

We ended up submitting two linear models based on PCA (100 components) transformed features of the final layer of the CLIPVisionModel (submission score = 43.262) and CLIPTextModel (submission score = 34.210). 

#### Code

In [ ]:
raise Exception("This code is not meant to be run. It is only meant to be used as a reference for the code used to generate the submission.")

# First test submission
subjects = [Subject(f"subj0{i}") for i in range(1, 9)]
CreateSubmission(subjects, procedure=CLIPVisionFinalLayerPCA100LinearRegression()).run()

# Second test submission
CreateSubmission(subjects, procedure=CLIPTextFinalLayerPCA100LinearRegression()).run()

### Third Submission - Vision Only Layer 7 PCA 200

For our third submission we used the 8-fold cross validation described above [Vision Only Model, PCA200, Linear Regression](#cv_vis_pca200_linreg)
. This submission resulted in our highest score to date (49.318).

#### Code

In [ ]:
raise Exception("This code is not meant to be run. It is only meant to be used as a reference for the code used to generate the submission.")

# Third submission
subjects = [Subject(f"subj0{i}") for i in range(1, 9)]
CreateSubmission(subjects, procedure=CLIPVisionLayer7PCA200LinearRegression()).run()

### Fourth Submission - Combined Model Layer 7 PCA 200 (PLANNED)

For our fourth submission we used the 8-fold cross validation described above [CLIP Text + Vision Model, PCA200, Linear Regression](#cv_txtvis_pca200_linreg). This submission STILL HAS TO BE DONE EXECUTE CODE BELOW.

#### Code

In [ ]:
raise Exception("This code is not meant to be run. It is only meant to be used as a reference for the code used to generate the submission.")

# Fourth submission
subjects = [Subject(f"subj0{i}") for i in range(1, 9)]
CreateSubmission(subjects, procedure=CLIPCombinedLayer7PCA200LinearRegression()).run()

In [ ]:
############## THIS ACTUALLY STILL HAS TO BE RUN ################
# Fourth submission
subjects = [Subject(f"subj0{i}") for i in range(1, 9)]
CreateSubmission(subjects, procedure=CLIPVisionLayer7PCA200LinearRegression()).run()